In [1]:
import pandas as pd
import numpy as np
import os
import json
import re
import random
from scipy.sparse import coo_matrix
from scipy.io import mmread, mmwrite, mminfo
# 用 base 环境，anaconda3 的pandas版本不对

### 生成上市公司、非上市公司、自然人三个列表

In [11]:
# 五种关系
subfirm = pd.read_csv('../上市公司子公司情况表/subsidiary_fullname_matched.csv',encoding='gb18030')
qinshu = pd.read_csv('../股权及亲属关系/qinshu_no_dup_person.csv',encoding='gb18030')
gaoguan = pd.read_csv('../高管关系/gaoguan_no_dup_person.csv',encoding='gb18030')
guanlian = pd.read_csv('../关联方交易/guanlian_no_dup_person.csv',encoding='gb18030')
gudong = pd.read_csv('../十大股东文件/gudong_not_fin.csv',encoding='gb18030')
# gudong = pd.read_csv('../十大流通股股东/gudong_no_dup_person.csv',encoding='gb18030')

#读取train_test数据集
# train_test = pd.read_csv('../train_test_x10.csv',encoding='gb18030')
stk_fullname = pd.read_csv('../stk_fullname.csv',encoding='gb18030')
fullname_id_dict = dict((stk_fullname.FullName[i],stk_fullname.Stkcd[i]) for i in stk_fullname.index)

print('每种关系的数量：子公司：%d；亲属：%d；高管：%d；关联交易：%d；股东关系：%d；共计%d' % 
      (len(subfirm), len(qinshu), len(gaoguan), len(guanlian), len(gudong),len(subfirm)+len(qinshu)+len(gaoguan)+len(guanlian)+len(gudong)) )

每种关系的数量：子公司：872305；亲属：16684；高管：844626；关联交易：686116；股东关系：540138；共计2959869


完整的上市公司列表来源于 code_for_CSMAR.xlsx，code_for_CSMAR 是由 会计信息质量-财务指标/all_corp_fin_data.xlsx 里面的上市公司提取出来的，就是财务指标完整的上市公司

In [12]:
n_listed_corp = 0
n_unlisted_corp = 0
n_people = 0

# 先生成所有上市公司的一个列表
code_for_CSMAR = pd.read_excel('../code_for_CSMAR.xlsx',encoding = 'utf-8')
listed_corp, unlisted_corp = set(code_for_CSMAR.code), set()
# 子公司里面有一些是上市公司
# unlisted_corp = set(subfirm.subsidiary)-set([str(i).zfill(6) for i in listed_corp])
for i in set(subfirm.subsidiary):
    try:
        # 把子公司里面的上市公司代码加到listed_corp集合中
        listed_corp.add(int(i))
    except:
        unlisted_corp.add(i)

# 高管关系中的manager和亲属关系中的person1和person2都是个人
people = set(list(gaoguan.manager)+list(qinshu.person1)+list(qinshu.person2))
print('子公司中有%d家上市公司,剔除,剩下的作为初始的非上市公司集合' % len(set(subfirm.subsidiary)-unlisted_corp))

子公司中有134家上市公司,剔除,剩下的作为初始的非上市公司集合


In [14]:
# 股东关系,holded不可能是个人，holder可能是个人、上市公司、非上市公司
last_name = r'^[赵钱孙李周吴郑王冯陈褚卫蒋沈韩杨朱秦尤许何吕施张孔曹严华金魏陶姜戚谢邹喻柏水窦章云苏潘葛奚范彭郎鲁韦昌马苗凤花方俞任袁柳酆鲍史唐费廉岑薛雷贺倪汤滕殷罗毕郝邬安常乐于时傅皮卞齐康伍余元卜顾孟平黄和穆萧尹姚邵湛汪祁毛禹狄米贝明臧计伏成戴谈宋茅庞熊纪舒屈项祝董梁杜阮蓝闵席季麻强贾路娄危江童颜郭梅盛林刁钟徐邱骆高夏蔡田樊胡凌霍虞支柯昝管卢莫经房裘缪干解应宗丁宣贲邓郁单杭洪包诸左石崔吉钮龚程嵇邢滑裴陆荣翁荀羊於惠甄麴家封芮羿储靳汲邴糜松井段富巫乌焦巴弓牧隗山谷车侯宓蓬全郗班仰秋仲伊宫宁仇栾暴甘钭厉戎祖武符刘景詹束叶幸司韶郜黎蓟薄印宿白怀蒲邰从鄂索咸籍赖卓蔺屠蒙池乔阴郁胥能苍双闻莘党翟谭贡劳逄姬申扶堵冉宰郦雍舄璩桑桂濮牛寿通边扈燕冀郏浦尚农温别庄晏柴瞿阎充慕连茹习宦艾鱼容向古易慎戈廖庾终暨居衡步都耿满弘匡国文寇广禄阙东殴殳沃利蔚越夔隆师巩厍聂晁勾敖融冷訾辛阚那简饶空曾毋沙乜养鞠须丰巢关蒯相查後荆红游竺权逯盖益桓公晋楚闫法汝鄢涂钦岳帅缑亢况后有商牟佘佴伯赏南宫墨哈谯笪年爱阳佟言福欧候劳原卿]'
firm_suffix = r'公司|有限|股份|企业|责任|集团|银行|证券|基金|保险|科技|资管|Ltd|LTD|INC|Inc|LIMITED|A/C|中国|Bank|平台|Corp|CORP|医院|欧洲|控股|大学|煤炭|地产|绿化|服饰|小学|船厂|发展|药业|城投|建投|实业|增长'
for i in gudong.index:
    holded = gudong.loc[i,'Stkcd']
    holder = gudong.loc[i,'shareholder']
    # 后半部分的数字股票代码是字符串，用type = int识别不出来
    try: # 判断是否是数字或者数字字符串
        holded = int(holded)
        listed_corp.add(holded)
    except:
        unlisted_corp.add(holded)
    
    try: # 判断是否是数字或者数字字符串
        holder = int(holder)
        listed_corp.add(holder)
    except:
        #if type(holder)==int or re.match('^[0-9]+',holder) is not None:
        if holder in unlisted_corp or holder in people:
            continue
        elif len(re.findall(firm_suffix,holder)) > 0: # 不要用re.match，有些识别不出来
            unlisted_corp.add(holder)
        elif len(re.findall(r'^[^0-9A-Za-z]+-[0-9]+',holder)) > 0:
            people.add(holder)
        elif len(holder)<=4 and len(re.findall(last_name,holder)) > 0:
            people.add(holder)
        else:
            unlisted_corp.add(holder)

In [15]:
# 关联关系
for i in guanlian.index:
    seller = guanlian.loc[i,'seller']
    buyer = guanlian.loc[i,'buyer']
    try:
        seller = int(seller)
        listed_corp.add(seller)
    except:
        if seller not in unlisted_corp and seller not in people:
            if len(re.findall(firm_suffix,seller)) > 0 :
                unlisted_corp.add(seller)
            elif len(seller)<=4 and len(re.findall(last_name,seller)) > 0:
                people.add(seller)
            elif len(re.findall(r'^[^0-9A-Za-z]+-[0-9]+',seller)) > 0:
                people.add(seller)
            else:
                unlisted_corp.add(seller)

    try:
        buyer = int(buyer)
        listed_corp.add(buyer)
    except:
        if buyer in unlisted_corp or buyer in people:
            continue
        elif len(re.findall(firm_suffix,buyer)) > 0:
            unlisted_corp.add(buyer)
        elif len(buyer)<=4 and len(re.findall(last_name,buyer)) > 0 :
            people.add(buyer)
        elif len(re.findall(r'^[^0-9A-Za-z]+-[0-9]+',buyer)) > 0:
            people.add(buyer)
        else:
            unlisted_corp.add(buyer)


In [18]:
n_listed_corp = len(listed_corp)
n_unlisted_corp = len(unlisted_corp)
n_people = len(people)

listed_corp = sorted(listed_corp)
people = sorted(people)
unlisted_corp = sorted(unlisted_corp)
print(n_listed_corp, n_unlisted_corp, n_people)
# 3月2日版本：4111 324764 212646

4111 324764 212646


写入 listed_firm.json，people.json，unlisted_firm.json

In [ ]:
'''with open('listed_firm.json','w') as f:
    json.dump(list(listed_corp),f,ensure_ascii=False)

with open('people.json','w') as f:
    json.dump(list(people),f,ensure_ascii=False)

with open('unlisted_firm.json','w') as f:
    json.dump(list(unlisted_corp),f,ensure_ascii=False)'''

In [ ]:
last_name = '赵、钱、孙、李、周、吴、郑、王、冯、陈、褚、卫、蒋、沈、韩、杨、朱、秦、尤、许、何、吕、施、张、孔、曹、严、华、金、魏、陶、姜、戚、谢、邹、喻、柏、水、窦、章、云、苏、潘、葛、奚、范、彭、郎、鲁、韦、昌、马、苗、凤、花、方、俞、任、袁、柳、酆、鲍、史、唐、费、廉、岑、薛、雷、贺、倪、汤、滕、殷、罗、毕、郝、邬、安、常、乐、于、时、傅、皮、卞、齐、康、伍、余、元、卜、顾、孟、平、黄、和、穆、萧、尹、姚、邵、湛、汪、祁、毛、禹、狄、米、贝、明、臧、计、伏、成、戴、谈、宋、茅、庞、熊、纪、舒、屈、项、祝、董、梁、杜、阮、蓝、闵、席、季、麻、强、贾、路、娄、危、江、童、颜、郭、梅、盛、林、刁、钟、徐、邱、骆、高、夏、蔡、田、樊、胡、凌、霍、虞、支、柯、昝、管、卢、莫、经、房、裘、缪、干、解、应、宗、丁、宣、贲、邓、郁、单、杭、洪、包、诸、左、石、崔、吉、钮、龚、程、嵇、邢、滑、裴、陆、荣、翁、荀、羊、於、惠、甄、麴、家、封、芮、羿、储、靳、汲、邴、糜、松、井、段、富、巫、乌、焦、巴、弓、牧、隗、山、谷、车、侯、宓、蓬、全、郗、班、仰、秋、仲、伊、宫、宁、仇、栾、暴、甘、钭、厉、戎、祖、武、符、刘、景、詹、束、叶、幸、司、韶、郜、黎、蓟、薄、印、宿、白、怀、蒲、邰、从、鄂、索、咸、籍、赖、卓、蔺、屠、蒙、池、乔、阴、郁、胥、能、苍、双、闻、莘、党、翟、谭、贡、劳、逄、姬、申、扶、堵冉、宰、郦、雍、舄、璩、桑、桂、濮、牛、寿、通、边、扈、燕、冀、郏、浦、尚、农、温、别、庄、晏、柴、瞿、阎、充、慕、连、茹、习、宦、艾、鱼、容、向、古、易、慎戈、廖、庾、终、暨、居、衡、步、都、耿、满、弘、匡、国、文、寇、广、禄、阙、东殴、殳、沃、利、蔚、越、夔、隆、师、巩、厍、聂、晁、勾、敖、融、冷、訾、辛、阚那、简、饶、空、曾、毋、沙、乜、养、鞠、须、丰、巢、关、蒯、相、查、後、荆、红游、竺、权、逯、盖、益、桓、公晋、楚、闫、法、汝、鄢、涂、钦岳、帅、缑、亢、况、后、有商、牟、佘、佴、伯、赏、南宫、墨、哈、谯、笪、年、爱、阳、佟言、福'
last_name = last_name.replace('、','')
last_name

In [ ]:
# 查看 people 集合中还有多少是公司的名称
for i in people:
    if re.match(r'^[^0-9A-Za-z]+-[0-9]+',i) is None and re.match(r'[^a-zA-Z]+',i) is not None:
        if len(i)>3:
            print(i)

### 读取实体列表

In [31]:
with open('listed_firm.json','r') as f:
    listed_corp = json.load(f)
with open('unlisted_firm.json', 'r') as f:
    unlisted_corp = json.load(f)
with open('people.json','r') as f:
    people = json.load(f)

n_listed_corp = len(listed_corp)
n_unlisted_corp = len(unlisted_corp)
n_people = len(people)
n_all = n_listed_corp + n_unlisted_corp + n_people

print(n_listed_corp, n_unlisted_corp, n_people)

# 3月2日版本：4111 324764 212646
# 5月30日版本：4111 324764 212646，和3月2日一样

all_body = listed_corp + unlisted_corp + people
print(len(all_body))

# all_body_dict 用来找 entity 的下标
all_body_dict = {}
for i in range(len(all_body)):
    all_body_dict[all_body[i]] = i

4111 324764 212646
541521


### 计算 one hop 关系

In [ ]:
# 亲戚关系
# 构造matrix by year
all_body = listed_corp + unlisted_corp + people

for year in range(2003,2022):
    for i in qinshu.index:
        cur_year = int(qinshu.loc[ i,'interval'])
        if cur_year != year:
            continue

        p1 = qinshu.loc[i,'person1']
        p2 = qinshu.loc[i,'person2']

        _row.append(all_body.index(p1))
        _col.append(all_body.index(p2))
        _data.append(1)

        _row.append(all_body.index(p2))
        _col.append(all_body.index(p1))
        _data.append(1)
    _row, _col, _data = np.array(_row), np.array(_col), np.array(_data)

    coo = coo_matrix((_data, (_row, _col)), shape= (n_all,n_all), dtype=np.int)
    mmwrite(os.path.join('mat_by_year', str(year), 'qinshu.mtx'),coo)
    _row, _col, _data = [], [], []
    
    print(year, coo.nnz, coo.shape,' qinshu done.')

In [ ]:
# 高管关系
# 构造matrix by year
all_body = listed_corp + unlisted_corp + people

for year in range(2003,2022):
    for i in gaoguan.index:
        cur_year = int(gaoguan.loc[i,'interval'])
        if cur_year != year:
            continue
        stk = gaoguan.loc[i,'Stkcd']
        manager = gaoguan.loc[i,'manager']

        _row.append(listed_corp.index(stk))
        _col.append(all_body.index(manager))
        _data.append(1)

    _row, _col, _data = np.array(_row), np.array(_col), np.array(_data)

    coo = coo_matrix((_data, (_row, _col)), shape= (n_listed_corp, n_all), dtype=np.int)
    
    mmwrite(os.path.join('mat_by_year', str(year), 'gaoguan.mtx'),coo)
    _row, _col, _data = [], [], []
    print(year, coo.nnz, coo.shape,'gaoguan done.')

In [ ]:
# 关联交易
_row, _col, _data = [], [], []
all_body = listed_corp + unlisted_corp + people
for year in range(2003,2022):
    for i in guanlian.index:
        cur_year = guanlian.loc[i,'interval']
        
        if year != cur_year:
            continue
        try:
            seller = int(guanlian.loc[i,'seller'])
        except:
            seller = guanlian.loc[i,'seller']
        try:
            buyer = int(guanlian.loc[i,'buyer'])
        except:
            buyer = guanlian.loc[i,'buyer']
        
        _row.append(all_body.index(seller))
        _col.append(all_body.index(buyer))
        _data.append(1)
        
        # 因为在计算元路径的时候，关联交易不管是买方是目标上市公司还是卖方是目标上市公司，都要算在一个矩阵里，这里做成对称的矩阵更好
        # 但是对称操作会使得有些数据重复，则导致 data 值大于1
        _row.append(all_body.index(buyer))
        _col.append(all_body.index(seller))
        _data.append(1)
    
    coo = coo_matrix((_data, (_row, _col)), shape = (len(all_body),len(all_body)), dtype=np.int)
    mmwrite(os.path.join('mat_by_year', str(year), 'guanlian.mtx'), coo)
    _row, _col, _data = [], [], []
    print(year, coo.nnz, coo.shape)


In [ ]:
# 子公司关系
_row, _col, _data = [], [], []
all_corp = listed_corp + unlisted_corp
all_body = listed_corp + unlisted_corp + people

# 构造matrix by year
for year in range(2003,2022):
    for i in subfirm.index:
        cur_year = int(subfirm.loc[i,'interval'])
        if cur_year != year:
            continue

        stkcd = subfirm.loc[i,'Stkcd']
        
        try: # 判断子公司是上市公司还是非上市公司
            sub = int(subfirm.loc[i,'subsidiary'])
        except:
            sub = subfirm.loc[i,'subsidiary']
        
        _row.append(listed_corp.index(stkcd))
        
        _col.append(all_corp.index(sub))
        
        _data.append(1)

    _row, _col, _data = np.array(_row), np.array(_col), np.array(_data)

    coo = coo_matrix((_data, (_row, _col)), shape= (n_listed_corp, n_listed_corp + n_unlisted_corp), dtype=np.int)
    # coo.todense()  # 通过toarray方法转化成密集矩阵(numpy.matrix)
    mmwrite(os.path.join('mat_by_year', str(year), 'subfirm.mtx'),coo)
    _row, _col, _data = [], [], []
    print(year, coo.nnz, coo.shape, 'subsidiary done.')

In [ ]:
# 股东关系
# all_corp = listed_corp + unlisted_corp
all_body = listed_corp + unlisted_corp + people

_row, _col, _data = [], [], []
for year in range(2003,2022):
    for i in gudong.index:
        cur_year = gudong.loc[i,'interval']
        if year != cur_year:
            continue
        holded = int(gudong.loc[i,'Stkcd'])
        try:
            holder = int(gudong.loc[i,'shareholder'])
        except:
            holder = gudong.loc[i,'shareholder']
        
        _row.append(listed_corp.index(holded))
        _col.append(all_body.index(holder))
        _data.append(1)
    coo = coo_matrix((_data, (_row, _col)), shape = (n_listed_corp, n_all), dtype=np.int)
    mmwrite(os.path.join('mat_by_year', str(year), 'gudong.mtx'),coo)
    _row, _col, _data = [], [], []
    # print(year,' gudong done.')
    print(year, coo.nnz, coo.shape)

### 计算相似度

In [35]:
with open('listed_firm.json','r') as f:
    listed_corp = json.load(f)
with open('unlisted_firm.json', 'r') as f:
    unlisted_corp = json.load(f)
with open('people.json','r') as f:
    people = json.load(f)

n_listed_corp = len(listed_corp)
n_unlisted_corp = len(unlisted_corp)
n_people = len(people)
n_all = n_listed_corp + n_unlisted_corp + n_people

print(n_listed_corp, n_unlisted_corp, n_people, n_all)

def trans(D):
    return coo_matrix((D.data, (D.col, D.row)), shape=(D.shape[1], D.shape[0]))
# 3124 254457 160165 417746

4111 324764 212646 541521


### 长度为2的路径

In [46]:
import meta_2rel_v2 # v2 太慢了

def listed_corp_slices(coo_mtx, n_listed_corp): # 对原始的矩阵进行行切片，以加快计算速度
    _row, _col, _data = [], [], []
    
    for row, col, data in zip(coo_mtx.row, coo_mtx.col, coo_mtx.data):
        if row < n_listed_corp:
            _row.append(row)
            _col.append(col)
            _data.append(data)
    return coo_matrix((_data, (_row, _col)), shape = (n_listed_corp, coo_mtx.shape[1]), dtype=np.int)


In [ ]:
# listed_firm - [HasTransaction] - all_body - [ShareholderOf] - listed_firm
# listed_firm - [ShareholderOf] - all_body - [HasTransaction] - listed_firm
'''数据量比较少'''
for year in range(2003,2022):
    # [listed_corp, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gudong.mtx'))
    
    # [all_corp, all_body]  -> [listed_corp, all_body]
    # gudong = listed_corp_slices(gudong, n_listed_corp=n_listed_corp)

    # [all_body, all_body]
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    # [all_body, all_body]  -> [listed_corp, all_body]
    guanlian = listed_corp_slices(guanlian, n_listed_corp=n_listed_corp)
    guanlian_T = trans(guanlian)
    gudong_guanlian = meta_2rel_v2.cal_metapath(gudong, guanlian_T, del_diagnoal=False)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gud_trans.mtx'), gudong_guanlian)
    print(year, gudong_guanlian.nnz, gudong_guanlian.shape)

In [ ]:
# listed_firm - [ShareholderOf] - all_body - [ShareholderOf] - listed_firm
for year in range(2003,2022):
    # [listed_corp, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gudong.mtx'))
    # [all_corp, all_body] -> [listed_corp, all_body]
    # gudong = listed_corp_slices(gudong, n_listed_corp=n_listed_corp)

    gudong_T = trans(gudong)

    gudong_gudong = meta_2rel_v2.cal_metapath(gudong, gudong_T, del_diagnoal=True)
    print(year, gudong_gudong.nnz, gudong_gudong.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gud_gud.mtx'), gudong_gudong)

In [ ]:
# listed_corp - [ManagerOf] - all_body(buyer) - [ShareholderOf] - listed_corp
'''数量比较少'''
for year in range(2003,2022):
    # [listed_corp, all_body]
    gaoguan = mmread(os.path.join('mat_by_year',str(year),'gaoguan.mtx'))
    
    # [listed_corp, all_body]
    gudong =  mmread(os.path.join('mat_by_year',str(year), 'gudong.mtx'))
    # [all_corp, all_body] -> [listed_corp, all_body]
    # gudong = listed_corp_slices(gudong, n_listed_corp=n_listed_corp)
    gudong_T = trans(gudong)

    mana_gudong = meta_2rel_v2.cal_metapath(gaoguan, gudong_T, del_diagnoal=False)
    print(year, mana_gudong.nnz, mana_gudong.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/mana_gud.mtx'), mana_gudong)


In [ ]:
# listed_corp - [ManagerOf] - people - [ManagerOf] - listed_corp
'''数量比较少'''
for year in range(2003,2022):
    # [listed_corp, all_body]
    gaoguan = mmread(os.path.join('mat_by_year',str(year),'gaoguan.mtx'))
    gaoguan_T = trans(gaoguan)
    mana_mana = meta_2rel_v2.cal_metapath(gaoguan, gaoguan_T, del_diagnoal=True)
    print(year, mana_mana.nnz, mana_mana.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/mana_mana.mtx'), mana_mana)


In [ ]:
# listed_corp - [HasTransaction] - all_body - [HasTransaction] - listed_corp
'''数量比较少'''
for year in range(2003, 2022):
    
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    guanlian = listed_corp_slices(guanlian, n_listed_corp=n_listed_corp)
    guanlian_T = trans(guanlian)
    guan_guan = meta_2rel_v2.cal_metapath(guanlian, guanlian_T, del_diagnoal=True)
    print(year, guan_guan.nnz, guan_guan.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/trans_trans.mtx'), guan_guan)


In [36]:
# listed_corp - [SubsidiaryOf] - all_body - [ShareholderOf] - listed_corp
'''数量比较少, 没有输出到文件中'''
for year in range(2003, 2022):
    # [listed_corp, all_corp]
    subfirm = mmread(os.path.join('mat_by_year',str(year),'subfirm.mtx'))
    # [listed_corp, all_body]
    gudong =  mmread(os.path.join('mat_by_year',str(year), 'gudong.mtx'))
    
    # gudong = listed_corp_slices(gudong, n_listed_corp=n_listed_corp)
    gudong_T = trans(gudong)
    
    sub_gudong = meta_2rel_v2.cal_metapath(subfirm, gudong_T, del_diagnoal=False)
    print(year, sub_gudong.nnz, sub_gudong.shape)
    

2003 65 (4111, 4111)
2004 85 (4111, 4111)
2005 80 (4111, 4111)
2006 91 (4111, 4111)
2007 81 (4111, 4111)
2008 76 (4111, 4111)
2009 90 (4111, 4111)
2010 91 (4111, 4111)
2011 123 (4111, 4111)
2012 106 (4111, 4111)
2013 107 (4111, 4111)
2014 118 (4111, 4111)
2015 124 (4111, 4111)
2016 156 (4111, 4111)
2017 160 (4111, 4111)
2018 152 (4111, 4111)
2019 158 (4111, 4111)
2020 167 (4111, 4111)
2021 155 (4111, 4111)


2023年5月30日

In [38]:
# listed_corp - [SubsidiaryOf] - all_body - [SubsidiaryOf] - listed_corp
'''数量比较少, 没有输出到文件中'''
for year in range(2003, 2022):
    # [listed_corp, all_corp]
    subfirm = mmread(os.path.join('mat_by_year',str(year),'subfirm.mtx'))
    subfirm_T = trans(subfirm)
    
    sub_sub = meta_2rel_v2.cal_metapath(subfirm, subfirm_T, del_diagnoal=False)
    print(year, sub_sub.nnz, sub_sub.shape)
    

2003 1124 (4111, 4111)
2004 1195 (4111, 4111)
2005 1257 (4111, 4111)
2006 1332 (4111, 4111)
2007 1431 (4111, 4111)
2008 1505 (4111, 4111)
2009 1625 (4111, 4111)
2010 1658 (4111, 4111)
2011 2170 (4111, 4111)
2012 2321 (4111, 4111)
2013 2397 (4111, 4111)
2014 2463 (4111, 4111)
2015 2698 (4111, 4111)
2016 2983 (4111, 4111)
2017 3484 (4111, 4111)
2018 3589 (4111, 4111)
2019 3789 (4111, 4111)
2020 3812 (4111, 4111)
2021 3843 (4111, 4111)


In [ ]:
# listed_corp - [SubsidiaryOf] - all_body - [SubsidiaryOf] - listed_corp


In [ ]:
# listed_corp - [SubsidiaryOf] - all_body - [HasTransactionOf] - listed_corp
'''数量比较少'''
for year in range(2003, 2022):
    # [listed_corp, all_corp]
    subfirm = mmread(os.path.join('mat_by_year',str(year),'subfirm.mtx'))
    # [all_body, all_body]
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    guanlian = listed_corp_slices(guanlian, n_listed_corp=n_listed_corp)
    guanlian_T = trans(guanlian)
    
    sub_guanlian = meta_2rel_v2.cal_metapath(subfirm, guanlian_T, del_diagnoal=False)
    print(year, sub_guanlian.nnz, sub_guanlian.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/sub_trans.mtx'), sub_guanlian)

    

### 长度为3的路径

In [ ]:
import meta_3rel_v1, meta_3rel_v2
# gudong: (listed_corp + unlisted_corp) * (listed_corp + unlisted_corp + people)

# first_quadrant 切到第一象限: 行往小了切，列往大了切
def first_quadrant_slices(coo_mtx, upper_bound, lower_bound):
    _row, _col, _data = [], [], []
    for r, c, d in zip(coo_mtx.row, coo_mtx.col, coo_mtx.data):
        if r < upper_bound and c >= lower_bound:
            _row.append(r)
            _col.append(c-lower_bound)
            _data.append(d)
    return coo_matrix((_data, (_row, _col)), shape = (upper_bound, coo_mtx.shape[1]-lower_bound), dtype=np.int)

In [ ]:
# listed_corp - (ShareholderOf) - person - (KindshipOf) - person - (ShareholderOf) - listed_corp
'''数量比较少'''
for year in range(2003,2022):
    # [all_body, all_body]
    qinshu = mmread(os.path.join('mat_by_year', str(year), 'qinshu.mtx'))
    # [listed_corp, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gudong.mtx'))
    # [all_corp, all_body] -> [listed_corp, all_body]
    # gudong = listed_corp_slices(gudong, n_listed_corp=n_listed_corp)

    # print(gudong.shape, qinshu.shape)
    gudong_T = trans(gudong)
    gu_qin_gu = meta_3rel_v2.cal_metapath(gudong, qinshu, gudong_T)
    print(year,gu_qin_gu.nnz, gu_qin_gu.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/gu_qin_gu.mtx'),gu_qin_gu)

In [ ]:
# listed_corp - [HasTransaction] - all_body - [HasTransaction] - all_body - [HasTransaction] - listed_corp
for year in range(2003,2022):
    guanlian = mmread(os.path.join('mat_by_year',str(year),'guanlian.mtx'))
    guanlian1 = listed_corp_slices(guanlian, n_listed_corp=n_listed_corp)
    guanlian_T = trans(guanlian1)
    guan_guan_guan = meta_3rel_v2.cal_metapath(guanlian1, guanlian, guanlian_T, del_diagnoal=False)
    print(year, guan_guan_guan.nnz, guan_guan_guan.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/trans_trans_trans.mtx'), guan_guan_guan)

In [ ]:
# listed_corp - [ManagerOf] - person - [KindshipOf] - person - [ManagerOf] - listed_corp
'''数量比较少'''
for year in range(2003,2022):
    # [all_body, all_body]
    qinshu = mmread(os.path.join('mat_by_year', str(year), 'qinshu.mtx'))
    # [listed_corp, all_body]
    gaoguan = mmread(os.path.join('mat_by_year',str(year), 'gaoguan.mtx'))
    gaoguan_T = trans(gaoguan)
    mana_qin_mana = meta_3rel_v2.cal_metapath(gaoguan, qinshu, gaoguan_T, del_diagnoal=False)
    print(year, mana_qin_mana.nnz, mana_qin_mana.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/mana_qin_mana.mtx'), mana_qin_mana)

In [ ]:
# listed_corp - [ManagerOf] - person - [KindshipOf] - person - [ShareholderOf] - listed_corp
'''数量比较少'''
for year in range(2003,2022):
    # [all_body, all_body]
    qinshu = mmread(os.path.join('mat_by_year', str(year), 'qinshu.mtx'))

    # [listed_corp, all_body]
    gaoguan = mmread(os.path.join('mat_by_year',str(year), 'gaoguan.mtx'))
    
    # [listed_corp, all_body]
    gudong = mmread(os.path.join('mat_by_year',str(year), 'gudong.mtx'))
    
    # gudong = listed_corp_slices(gudong, n_listed_corp = n_listed_corp)
    
    gudong_T = trans(gudong)
    
    # 股东
    mana_qin_gud = meta_3rel_v2.cal_metapath(gaoguan, qinshu, gudong_T, del_diagnoal=False)
    print(year, mana_qin_gud.nnz, mana_qin_gud.shape)
    mmwrite(os.path.join('mat_by_year',str(year),'metapath/mana_qin_gud.mtx'), mana_qin_gud)

#### 统计各个meta-path的数量:

In [ ]:
'''
P1: gud_gud
P2: gu_qin_gu
P3: gud_trans
P4: mana_gud
P5: mana_mana
P6: mana_qin_gud
P7: mana_qin_mana
P8: sub_trans
P9: trans_trans
P10: trans_trans_trans
'''

def path_stats(PATH_NAME):
    N_by_year = []
    for year in range(2003,2022):
        path = mmread(os.path.join('mat_by_year',str(year),f'metapath/{PATH_NAME}.mtx'))
        N_by_year.append(path.nnz)
    return N_by_year, np.sum(N_by_year)

N_by_year, n_agg = path_stats(PATH_NAME='trans_trans_trans')
print(N_by_year, n_agg)


查看 subsidiary 和 shareholder 的实体重复率

In [16]:
len(set(gudong.shareholder.tolist()) & set(subfirm.subsidiary.tolist()))

708

In [17]:
len(set(gudong.shareholder.tolist())), len(set(subfirm.subsidiary.tolist()))

(120278, 184869)

In [11]:
subfirm

,Stkcd,interval,subsidiary
0,2,2003,成都万科物业管理有限公司
1,2,2003,天津万科房地产有限公司
2,2,2003,天津万科兴业发展有限公司
3,2,2003,天津万科住宅发展有限公司
4,2,2003,天津万科物业管理有限公司
...,...,...,...
872300,900956,2019,阿拉山口东贝洁能有限公司
872301,900956,2019,东贝国际贸易有限公司
872302,900956,2019,东贝机电(江苏)有限公司
872303,900956,2019,东贝(武汉)科技创新有限公司


In [ ]:
mat_a = coo_matrix(np.array([
                    [1,0,0,0,1,0,0],
                    [1,1,0,0,0,0,1],
                ]))
mat_b = coo_matrix(np.array([
                    [1,1],
                    [0,0],
                    [0,0],
                    [0,0],
                    [1,0],
                    [0,0],
                    [0,1]
                ]))
mat_c = coo_matrix(np.array([
                    [1,1,1],
                    [0,1,1],
                ]))
mat_d = coo_matrix(np.array([
                    [1,1,0],
                    [0,1,0],
                    [0,0,1]
                ]))
'''coo = cal_metapath(mat_c, mat_d,[0,1,2])
coo_den = coo.todense()
coo_den'''
mat_a_T = trans(mat_a)
mat_a_T = mat_a_T.todense()
a = np.array([
  [1,0,0,0,1,0,0],
  [1,1,0,0,0,0,1],
])
b = np.array([
  [0,0,1,0,1,0,1],
  [1,1,0,1,0,1,0],
])
np.matmul(a, b.T)
np.matmul(b, a.T)

outdegree = np.array([list(mat_b.row).count(i) for i in range(mat_a.shape[1])])
print(outdegree)
outdegree_reverse = np.array([list(mat_a.col).count(i) for i in range(mat_b.shape[0])])
np.sum(1/2 * 1/outdegree[[0,4]])




In [ ]:
import torch
trans_trans_trans = mmread('mat_by_year/2018/metapath/trans_trans.mtx')
torch.from_numpy(trans_trans_trans.todense())
trans_trans_trans.data.shape

In [ ]:
import meta_3rel_v2, meta_3rel_v1, meta_4rel_v2

ab = meta_2rel_v2.cal_metapath(mat_a, mat_b, del_diagnoal=False)
ab = ab.todense()

ac = meta_3rel_v2.cal_metapath(mat_a, mat_b, mat_c, del_diagnoal=False)
ac = ac.todense()
ac

ad = meta_4rel_v2.cal_metapath(mat_a, mat_b, mat_c, mat_d, del_diagnoal=False)
ad = ad.todense()
ad

In [ ]:
92+134+152+198+198+68+218+84+128+36+48

In [25]:
import itertools
def find_permutations(elements, sample_num):
    permutations = list(itertools.product(elements, repeat=sample_num))
    return permutations

# 示例输入
n = 6
elements = ['shareholderOf','hasTransaction','managerOf','subsidiaryOf','kinshipOf']

result_3 = find_permutations(elements, 3)
result_3

[('shareholderOf', 'shareholderOf', 'shareholderOf'),
 ('shareholderOf', 'shareholderOf', 'hasTransaction'),
 ('shareholderOf', 'shareholderOf', 'managerOf'),
 ('shareholderOf', 'shareholderOf', 'subsidiaryOf'),
 ('shareholderOf', 'shareholderOf', 'kinshipOf'),
 ('shareholderOf', 'hasTransaction', 'shareholderOf'),
 ('shareholderOf', 'hasTransaction', 'hasTransaction'),
 ('shareholderOf', 'hasTransaction', 'managerOf'),
 ('shareholderOf', 'hasTransaction', 'subsidiaryOf'),
 ('shareholderOf', 'hasTransaction', 'kinshipOf'),
 ('shareholderOf', 'managerOf', 'shareholderOf'),
 ('shareholderOf', 'managerOf', 'hasTransaction'),
 ('shareholderOf', 'managerOf', 'managerOf'),
 ('shareholderOf', 'managerOf', 'subsidiaryOf'),
 ('shareholderOf', 'managerOf', 'kinshipOf'),
 ('shareholderOf', 'subsidiaryOf', 'shareholderOf'),
 ('shareholderOf', 'subsidiaryOf', 'hasTransaction'),
 ('shareholderOf', 'subsidiaryOf', 'managerOf'),
 ('shareholderOf', 'subsidiaryOf', 'subsidiaryOf'),
 ('shareholderOf', 's

In [32]:
# 路径长度为2
elements = ['shareholderOf','hasTransaction','managerOf','subsidiaryOf']

result_2 = find_permutations(elements, 2)
for path in result_2:
    print(f'(C1, {path[0]}, C3,{path[1]}, C2)')

(C1, shareholderOf, C3,shareholderOf, C2)
(C1, shareholderOf, C3,hasTransaction, C2)
(C1, shareholderOf, C3,managerOf, C2)
(C1, shareholderOf, C3,subsidiaryOf, C2)
(C1, hasTransaction, C3,shareholderOf, C2)
(C1, hasTransaction, C3,hasTransaction, C2)
(C1, hasTransaction, C3,managerOf, C2)
(C1, hasTransaction, C3,subsidiaryOf, C2)
(C1, managerOf, C3,shareholderOf, C2)
(C1, managerOf, C3,hasTransaction, C2)
(C1, managerOf, C3,managerOf, C2)
(C1, managerOf, C3,subsidiaryOf, C2)
(C1, subsidiaryOf, C3,shareholderOf, C2)
(C1, subsidiaryOf, C3,hasTransaction, C2)
(C1, subsidiaryOf, C3,managerOf, C2)
(C1, subsidiaryOf, C3,subsidiaryOf, C2)


In [37]:
len(result_2)

16